FINAL TEST
insider transactions, trading volume, analyst ratings, price fluctuations, market capitalization
Insider Transactions: Located under the "Holders" section, it lists the buys and sells by company insiders.
Trading Volume: Found on the "Summary" and "Historical Data" pages, it shows the number of shares traded during a given period.
Analyst Ratings: Located in the "Analysis" section, it provides the ratings and price targets set by financial analysts.

In [44]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

load_dotenv()

username = os.getenv('username')
password = os.getenv('password')
host = os.getenv('host')
port = os.getenv('port')
database = os.getenv('database')

engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

# Set up the Selenium driver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)



In [45]:
load_dotenv()

username = os.getenv('username')
password = os.getenv('password')
host = os.getenv('host')
port = os.getenv('port')
database = os.getenv('database')

engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

In [46]:
# Set up the Selenium driver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

In [47]:
def extract_table_data(table_element):
    rows = table_element.find_elements(By.TAG_NAME, 'tr')
    headers = [header.text for header in rows[0].find_elements(By.TAG_NAME, 'th')]
    data = [[td.text for td in row.find_elements(By.TAG_NAME, 'td')] for row in rows[1:]]
    return pd.DataFrame(data, columns=headers)

def dataframe_to_sql(df, table_name, engine):
    df.to_sql(name=table_name, con=engine, if_exists='append', index=False)

In [48]:
# List of symbols to iterate over
symbols = ["JPM", "GS", "C", "JLL", "DIS", "TPR", "F", "XOM", "AAPL", "AMZN", "PFE", "MRK"]

In [49]:
# Collect data for each symbol
all_insider_transactions = []
all_trading_volumes = []
all_earnings_estimates = []
all_eps_trends = []
all_eps_revisions = []
all_growth_estimates = []

In [50]:
for symbol in symbols:
    # Insider Transactions
    url = f'https://finance.yahoo.com/quote/{symbol}/insider-transactions'
    driver.get(url)
    time.sleep(10)
    insider_table = driver.find_element(By.CLASS_NAME, 'svelte-1s2g2l0')
    insider_transactions_df = extract_table_data(insider_table)
    insider_transactions_df['Symbol'] = symbol
    all_insider_transactions.append(insider_transactions_df)

    # Trading Volume
    url = f'https://finance.yahoo.com/quote/{symbol}/history?filter=history'
    driver.get(url)
    time.sleep(6)
    trading_volume_table = driver.find_element(By.CSS_SELECTOR, '.table.svelte-ewueuo')
    trading_volume_df = extract_table_data(trading_volume_table)
    trading_volume_df['Symbol'] = symbol
    all_trading_volumes.append(trading_volume_df)

    # Analyst Ratings
    url = f'https://finance.yahoo.com/quote/{symbol}/analysis'
    driver.get(url)
    time.sleep(10)
    categories = {
        'earningsEstimate': all_earnings_estimates,
        'epsTrend': all_eps_trends,
        'epsRevisions': all_eps_revisions,
        'growthEstimate': all_growth_estimates
    }
    for category, container in categories.items():
        section_element = driver.find_element(By.XPATH, f'//section[@data-testid="{category}"]')
        df = extract_table_data(section_element)
        df['Symbol'] = symbol
        container.append(df)

In [ ]:
# Concatenate all dataframes for each category and upload to SQL
dataframe_to_sql(pd.concat(all_insider_transactions, ignore_index=True), 'insider_transactions')
dataframe_to_sql(pd.concat(all_trading_volumes, ignore_index=True), 'trading_volume')
dataframe_to_sql(pd.concat(all_earnings_estimates, ignore_index=True), 'earnings_estimate')
dataframe_to_sql(pd.concat(all_eps_trends, ignore_index=True), 'eps_trend')
dataframe_to_sql(pd.concat(all_eps_revisions, ignore_index=True), 'eps_revisions')
dataframe_to_sql(pd.concat(all_growth_estimates, ignore_index=True), 'growth_estimate')

In [ ]:
# Close the Selenium driver
driver.quit()

In [31]:
# Insider Transactions for each symbol
for symbol in symbols:
    url = f'https://finance.yahoo.com/quote/{symbol}/insider-transactions'
    driver.get(url)
    time.sleep(10)  # Adjust with WebDriverWait as needed
    insider_table = driver.find_element(By.CLASS_NAME, 'svelte-1s2g2l0')
    insider_transactions_df = extract_table_data(insider_table)
    insider_transactions_df['Symbol'] = symbol



In [32]:
# Loop over symbols and scrape trading volume
for symbol in symbols:
    url = f'https://finance.yahoo.com/quote/{symbol}/history?filter=history'
    driver.get(url)
    time.sleep(6)  # Adjust with WebDriverWait as needed
    
    trading_volume_table = driver.find_element(By.CSS_SELECTOR, '.table.svelte-ewueuo')
     = extract_table_data(trading_volume_table)

In [33]:
def scrape_ratings_category(symbols, category):
    ratings_data = []
    for symbol in symbols:
        url = f'https://finance.yahoo.com/quote/{symbol}/analysis'
        driver.get(url)
        time.sleep(10)  # Ensure the page has loaded
        section_element = driver.find_element(By.XPATH, f'//section[@data-testid="{category}"]')
        df = extract_table_data(section_element)
        df['Symbol'] = symbol
        ratings_data.append(df)
    return pd.concat(ratings_data, ignore_index=True)


In [ ]:
symbols = ["JPM", "GS", "C", "JLL", "DIS", "TPR", "F", "XOM", "AAPL", "AMZN", "PFE", "MRK"]
insider_transactions_df = scrape_insider_transactions(symbols)
trading_volume_df = scrape_trading_volume(symbols)
earnings_estimate_df = scrape_ratings_category(symbols, 'earningsEstimate')
eps_trend_df = scrape_ratings_category(symbols, 'epsTrend')
eps_revisions_df = scrape_ratings_category(symbols, 'epsRevisions')
growth_estimate_df = scrape_ratings_category(symbols, 'growthEstimate')

In [34]:
# Close the Selenium driver
driver.quit()

In [ ]:
dataframe_to_sql(insider_transactions_df, 'insider_transactions', engine)